<a href="https://colab.research.google.com/github/danilotpnta/UN-General-Debate-Analysis-SDGs/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UN-General-Debate-Analysis-SDGs

This project analyzes the UN General Debate Corpus from 1970 to 2023. It includes exploratory data analysis (EDA), predictive modeling, and data visualizations focusing on uncovering insights from political speeches and their connection to global challenges.

The project is divided into the following sections:

1. Data Preprocessing
2. Exploratory Data Analysis
3. Predictive Modeling
4. Data Visualization

## If running from Google Colab

In [12]:
# Clone your GitHub repository
!git clone https://github.com/danilotpnta/UN-General-Debate-Analysis-SDGs.git

# Navigate to the repository folder
%cd /content/UN-General-Debate-Analysis-SDGs

Cloning into 'UN-General-Debate-Analysis-SDGs'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 21 (delta 7), reused 11 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 6.96 KiB | 6.96 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/UN-General-Debate-Analysis-SDGs


## Data Preprocessing

#### Donwloading the dataset

In [13]:
from utils.dataverse_downloader import download_dataset

# Call the function to download all files
download_dataset()

Raw_PDFs_1946-1969.tgz: 100%|██████████| 1.49G/1.49G [01:44<00:00, 15.3MB/s]
Raw_PDFs_1970-1990.tgz: 100%|██████████| 2.03G/2.03G [02:21<00:00, 15.3MB/s]
Raw_PDFs_1991-2022.tgz: 100%|██████████| 280M/280M [00:17<00:00, 16.4MB/s]
README.txt: 100%|██████████| 1.49k/1.49k [00:00<00:00, 1.43MB/s]
Speakers_by_session.xlsx: 100%|██████████| 456k/456k [00:00<00:00, 3.16MB/s]
UNGDC_1946-2023.tgz: 100%|██████████| 66.2M/66.2M [00:04<00:00, 15.8MB/s]


## Exploratory Data Analysis
